In [1]:
using TensorKit
import TensorKit.×
# include("./iPEPS_fSU2/iPEPS.jl")
# include("./CTMRG_fSU2/CTMRG.jl")
@show pkgversion(TensorKit)

pkgversion(TensorKit) = v"0.12.2"


v"0.12.2"

In [2]:
# pspace = GradedSpace{fSU₂}((-1, 0) => 1, (0, 1 // 2) => 1, (1, 0) => 1)
pspace = GradedSpace{fSU₂}((0 => 2), (1//2 => 1))
aspacel = GradedSpace{fSU₂}((0 => 1), (1//2 => 1))
aspacet = GradedSpace{fSU₂}((0 => 1), (1//2 => 2))
aspacer = GradedSpace{fSU₂}((0 => 2), (1//2 => 2))
aspaceb = GradedSpace{fSU₂}((0 => 1), (1//2 => 1), (1 => 1))
# aspacel = aspacer = aspacet = aspaceb = pspace
@show pspace
nothing

pspace = Vect[FermionSpin]((0, 0)=>2, (1/2, 1)=>1)


In [7]:
a = id(pspace)
@show a
b = a * 2
c = sqrt(b)
@show c
b1 = c * c
@assert b1 ≈ b
@tensor b2[x, y] := c[x, in] * c[y, in]
@assert b2 ≈ b
nothing

a = TensorMap(Vect[FermionSpin]((0, 0)=>2, (1/2, 1)=>1) ← Vect[FermionSpin]((0, 0)=>2, (1/2, 1)=>1)):
* Data for fusiontree FusionTree{FermionSpin}(((0, 0),), (0, 0), (false,), ()) ← FusionTree{FermionSpin}(((0, 0),), (0, 0), (false,), ()):
 1.0  0.0
 0.0  1.0
* Data for fusiontree FusionTree{FermionSpin}(((1/2, 1),), (1/2, 1), (false,), ()) ← FusionTree{FermionSpin}(((1/2, 1),), (1/2, 1), (false,), ()):
 1.0

c = TensorMap(Vect[FermionSpin]((0, 0)=>2, (1/2, 1)=>1) ← Vect[FermionSpin]((0, 0)=>2, (1/2, 1)=>1)):
* Data for fusiontree FusionTree{FermionSpin}(((0, 0),), (0, 0), (false,), ()) ← FusionTree{FermionSpin}(((0, 0),), (0, 0), (false,), ()):
 1.4142135623730951 + 0.0im                 0.0 + 0.0im
                0.0 + 0.0im  1.4142135623730951 + 0.0im
* Data for fusiontree FusionTree{FermionSpin}(((1/2, 1),), (1/2, 1), (false,), ()) ← FusionTree{FermionSpin}(((1/2, 1),), (1/2, 1), (false,), ()):
 1.4142135623730951 + 0.0im



SpaceMismatch{String}: SpaceMismatch("(Vect[FermionSpin]((0, 0)=>2, (1/2, 1)=>1) ⊗ Vect[FermionSpin]((0, 0)=>2, (1/2, 1)=>1)') ← ProductSpace{GradedSpace{ProductSector{Tuple{SU2Irrep, FermionParity}}, TensorKit.SortedVectorDict{ProductSector{Tuple{SU2Irrep, FermionParity}}, Int64}}, 0}() ≠ Vect[FermionSpin]((0, 0)=>2, (1/2, 1)=>1) ← Vect[FermionSpin]((0, 0)=>2, (1/2, 1)=>1)")

In [ ]:
a = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
@show space(a)  # [l, t, p; r, b]
@show space(a') # [r, b; l, t, p]

Xl, vl = leftorth(a, ((1, 2, 5), (3, 4)))
@show space(Xl)
@show space(vl)
nothing

In [ ]:
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# pspace = Rep[ℤ₂]((1 => 1), (0 => 2))
aspacel = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspacer = Rep[ℤ₂×SU₂]((0, 0) => 2, (0, 1) => 2)
aspacer = aspacel
aspacet = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspaceb = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 3 // 2) => 2)
aspaceb = aspacet

# A = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
# Ms = [A, A, A, A]
# ipeps = iPEPS(Ms, 2, 2)
# Abar = bar(A)
# @show space(Abar)
# gate1 = swap_gate(space(Abar)[4], space(Abar)[5])
# @show space(gate1)
# gate2 = swap_gate(space(Abar)[1], space(Abar)[2])
# @show space(gate2)
# gate3 = swap_gate(space(A)[4], space(A)[5])
# @show space(gate3)
# gate4 = swap_gate(space(A)[1], space(A)[2])
# @show space(gate4)
# gate5 = swap_gate(space(A)[1], space(Abar)[2])
# @show space(gate5)
@assert pspace isa ElementarySpace
@show fieldnames(GradedSpace)

In [ ]:
A = id(pspace)
@show A
@tensor x = A[a, a]
@show x
nothing

In [ ]:
include("./simple_update_fSU2/Models.jl")
para = Dict{Symbol,Any}()
para[:t] = 1.0
para[:U] = 8
para[:τlis] = [0.01, 0.03, 0.05, 0.1, 0.2, 0.3]
para[:Dk] = 10  # Dkept in the simple udate

pspace = GradedSpace{fSU₂}((0 => 2), (1 // 2 => 1))
aspacelr = pspace
aspacetb = pspace
Lx = 2
Ly = 2
ipeps = iPEPSΓΛ(pspace, aspacelr, aspacetb, Lx, Ly; dtype=ComplexF64)

τlis = para[:τlis]
t = para[:t]
U = para[:U]
Dk = para[:Dk]
Nit = length(τlis)
hij = Hubbard_hij(t, U)
τ = τlis[1]
gateNN = exp(-τ * hij)

@show space(ipeps[1, 1].Γ)
@show space(ipeps[1, 1].l)
@show space(ipeps[1, 1].t)
@show space(ipeps[1, 1].r)
@show space(ipeps[1, 1].b)
nothing

In [ ]:
pspace = GradedSpace{fSU₂}((0 => 2), (1 // 2 => 1))
aspacel = GradedSpace{fSU₂}((0 => 2), (1 // 2 => 1))
aspacer = aspacel
aspacet = GradedSpace{fSU₂}((0 => 2), (1 // 2 => 1))
aspaceb = aspacet
A = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
Ms = [A, A, A, A]
ipeps = iPEPS(Ms, 2, 2)

envs = iPEPSenv(ipeps)
χ = 10
Nit = 2
nothing

In [ ]:
x = 1
Ly = ipeps.Ly
# 存储更新右侧环境需要的投影算符，矩阵第一列是上半部分proj ∇, 第二列是下半部分proj Δ
proj_List = Matrix{TensorMap}(undef, Ly, 2)
# 误差列表
error_List = Vector{Float64}(undef, Ly)
# ----------------- 先求proj ---------------------
for yy in 1:Ly
    projup, projdn, ϵ = get_proj_update_LR(ipeps, envs, x, yy, χ; dir="right")
    proj_List[yy, 1] = projup
    proj_List[yy, 2] = projdn
    error_List[yy] = ϵ
end
# # ------------------ 再更新环境 ----------------------
# for yy in 1:Ly
#     if yy == 1
#         apply_proj_right!(ipeps, envs, proj_List[Ly, 2], proj_List[yy, 1], x, yy)
#         apply_proj_rtCorner_updateR!(envs, proj_List[Ly, 1], x, 1)
#         apply_proj_rbCorner_updateR!(envs, proj_List[1, 2], x, 1)
#     else
#         apply_proj_right!(ipeps, envs, proj_List[yy-1, 2], proj_List[yy, 1], x, yy)
#         apply_proj_rtCorner_updateR!(envs, proj_List[yy-1, 1], x, yy)
#         apply_proj_rbCorner_updateR!(envs, proj_List[yy, 2], x, yy)
#     end
# end

In [ ]:
apply_proj_right!(ipeps, envs, proj_List[Ly, 2], proj_List[1, 1], x, 1)
nothing

In [ ]:
apply_proj_rtCorner_updateR!(envs, proj_List[Ly, 1], x, 1)
apply_proj_rbCorner_updateR!(envs, proj_List[1, 2], x, 1)
apply_proj_right!(ipeps, envs, proj_List[1, 2], proj_List[2, 1], x, 2)
nothing